# 🧪 전체 파이프라인 통합 테스트

이 노트북은 전체 학습/추론 파이프라인의 통합 동작을 테스트합니다:
- 데이터셋, 모델, 학습, 추론 모듈 연동
- 주요 입출력 및 로그 확인

In [ ]:
# [1] 프로젝트 루트 디렉토리로 이동
import os                                                   # OS 모듈 임포트
import sys                                                  # 시스템 모듈 임포트
os.chdir("../../../")                                       # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())                      # 현재 디렉토리 출력

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [3]:
# [2] 폰트 설정 및 경고 억제

# 경고 메시지 억제용 모듈 임포트
#!sudo apt -get install -y fonts-nanum  # 나눔폰트 설치 (필요시 주석 해제) 설치 후 vscode 재시작 필요
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 적용 및 시각화 환경 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔고딕 폰트 경로 및 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 폰트 등록 및 설정
fe = fm.FontEntry(fname=font_path, name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# 글자 겹침 방지를 위한 레이아웃 설정
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.titlepad'] = 20

In [4]:
# [3] 라이브러리 및 유틸리티 임포트
# 파이프라인, 설정, 로거 등 필요한 모듈 임포트
import torch  # 파이토치 임포트
from src.pipeline.full_pipeline import run_full_pipeline  # 전체 파이프라인 함수 임포트
from src.utils.common import load_yaml  # 설정 로드 함수 임포트
from src.logging.unit_test_logger import create_test_logger  # 단위 테스트 로거 생성 함수 임포트
test_logger = create_test_logger("pipeline_integration")  # 단위 테스트 로거 초기화

📝 단위 테스트 시작: pipeline_integration
📝 로그 디렉토리: notebooks/modular/unit_tests/pipeline_integration/20250906_000831


In [5]:
# [4] GPU 자동 체크
# GPU 사용 가능 여부 확인
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

✅ GPU 사용 가능: NVIDIA GeForce RTX 4090


In [10]:
# [5] 전체 파이프라인 실행 테스트
# 설정 파일 경로를 전달하여 전체 파이프라인 실행
try:  # 예외 처리 시작
    config_path = "configs/train_highperf.yaml"  # 설정 파일 경로
    cfg = load_yaml(config_path)  # 설정 파일 로드 (확인용)
    print(f"✅ 설정 파일 로드 성공: {config_path}")
    print(f"모델: {cfg['model']['name']}, 에포크: {cfg['train']['epochs']}")
    
    # 전체 파이프라인 실행 (설정 파일 경로 전달)
    result = run_full_pipeline(config_path, skip_training=True)  # 테스트이므로 학습 건너뛰기
    print(f'✅ 파이프라인 실행 결과: {result}')  # 결과 출력
    
    # 테스트 결과 저장
    test_result = {
        'pipeline_result': result,
        'config_file': config_path,
        'skip_training': True,
        'status': 'success'
    }
    test_logger.save_test_result('pipeline_integration_test', test_result)
    test_logger.finalize_test()
    print('✅ 파이프라인 통합 테스트 완료')
except Exception as e:  # 예외 발생 시
    print(f'⚠️ 파이프라인 실행 실패: {e}')  # 에러 메시지 출력
    # 에러 발생 시에도 로거 정리
    try:
        test_logger.save_test_result('pipeline_integration_test', {'status': 'failed', 'error': str(e)})
        test_logger.finalize_test()
    except:
        pass

✅ 설정 파일 로드 성공: configs/train_highperf.yaml
모델: swin_base_384, 에포크: 15
2025-09-06 00:14:58 | 🚀 [PIPELINE] Full pipeline started
2025-09-06 00:14:58 | 📋 Config: configs/train_highperf.yaml
2025-09-06 00:14:58 | ⚙️ Skip training: True
2025-09-06 00:14:58 | ⏭️ [STAGE 1] Training skipped
2025-09-06 00:14:58 | ============================================================
2025-09-06 00:14:58 | 🔍 [STAGE 2] FINDING TRAINING RESULTS
2025-09-06 00:14:58 | ============================================================
2025-09-06 00:14:58 | 📁 Found fold results: experiments/train/20250906/v094-swin-highperf/fold_results.yaml
2025-09-06 00:14:58 | ============================================================
2025-09-06 00:14:58 | 🔮 [STAGE 3] HIGH-PERFORMANCE INFERENCE
2025-09-06 00:14:58 | ============================================================
2025-09-06 00:14:58 | [BOOT] high-performance inference pipeline started
2025-09-06 00:14:58 | [BOOT] device=cuda
2025-09-06 00:14:58 | [DATA] loaded test d

TTA Inference: 100%|██████████| 99/99 [00:21<00:00,  4.58it/s]


Processing model 2/5...


TTA Inference: 100%|██████████| 99/99 [00:21<00:00,  4.53it/s]


Processing model 3/5...


...

Processing model 4/5...


TTA Inference: 100%|██████████| 99/99 [00:21<00:00,  4.52it/s]


Processing model 5/5...


TTA Inference: 100%|██████████| 99/99 [00:21<00:00,  4.52it/s]

2025-09-06 00:20:33 | [SUCCESS] Inference completed | output: submissions/20250906/v094-swin-highperf_ensemble_20250906_0014.csv
2025-09-06 00:20:33 | [RESULT] Prediction distribution:
2025-09-06 00:20:33 | Class 0: 200 samples (6.4%)
2025-09-06 00:20:33 | Class 1: 92 samples (2.9%)
2025-09-06 00:20:33 | Class 2: 200 samples (6.4%)
2025-09-06 00:20:33 | Class 3: 187 samples (6.0%)
2025-09-06 00:20:33 | Class 4: 214 samples (6.8%)
2025-09-06 00:20:33 | Class 5: 200 samples (6.4%)
2025-09-06 00:20:33 | Class 6: 207 samples (6.6%)
2025-09-06 00:20:33 | Class 7: 219 samples (7.0%)
2025-09-06 00:20:33 | Class 8: 200 samples (6.4%)
2025-09-06 00:20:33 | Class 9: 200 samples (6.4%)
2025-09-06 00:20:33 | Class 10: 205 samples (6.5%)
2025-09-06 00:20:33 | Class 11: 190 samples (6.1%)
2025-09-06 00:20:33 | Class 12: 202 samples (6.4%)
2025-09-06 00:20:33 | Class 13: 153 samples (4.9%)
2025-09-06 00:20:33 | Class 14: 71 samples (2.3%)
2025-09-06 00:20:33 | Class 15: 200 samples (6.4%)
2025-09-06 

## 📊 파이프라인 통합 테스트 결과 요약

### ✅ 테스트 성공 항목
1. **설정 파일 로드**: `configs/train_highperf.yaml` 성공적 로드
2. **파이프라인 단계**: 학습 건너뛰기, 결과 찾기, 고성능 추론 실행
3. **모델 앙상블**: 5개 모델의 TTA 추론 완료
4. **제출 파일**: 생성 및 저장 완료

### 📈 성능 지표
- **실행 시간**: 약 5분 37초 (337초)
- **처리 이미지**: 3,140개 테스트 이미지
- **TTA 증강**: 각 모델당 3회 증강 적용
- **앙상블 모델**: 5개 폴드 모델 사용

### 📁 생성된 파일 구조
```
logs/infer/
└── infer_highperf_YYYYMMDD_HHMM.log

submissions/YYYYMMDD/
├── submission_highperf_v094_HHMM.csv
└── prediction_details_v094_HHMM.json
```

### 🔧 파이프라인 구성 요소
- **데이터 로더**: HighPerfDocClsDataset (3,140 이미지)
- **모델**: Swin Transformer Base 384px
- **TTA**: 다중 증강 기법 적용
- **앙상블**: 5-fold 교차 검증 모델 평균
- **출력**: CSV 제출 파일 + JSON 상세 정보

In [11]:
# [6] 실제 생성된 파일 구조 확인
import os
import glob
from datetime import datetime

print("🔍 파이프라인 실행 결과 파일 확인\n")

# 1. 추론 로그 파일 확인 (실제 패턴)
log_pattern = "logs/infer/infer_highperf_*.log"
log_files = glob.glob(log_pattern)
print(f"📄 추론 로그 파일 ({len(log_files)}개):")
for log_file in sorted(log_files)[-3:]:  # 최근 3개만 표시
    size = os.path.getsize(log_file) / 1024  # KB
    mtime = datetime.fromtimestamp(os.path.getmtime(log_file))
    print(f"   └── {os.path.basename(log_file)} ({size:.1f} KB, {mtime.strftime('%Y-%m-%d %H:%M:%S')})")

# 2. 제출 파일 디렉토리 확인
print(f"\n📦 제출 파일 디렉토리들:")
submission_dirs = glob.glob("submissions/202509*")
for sub_dir in sorted(submission_dirs)[-3:]:  # 최근 3개만 표시
    print(f"   ├── {sub_dir}")
    if os.path.exists(sub_dir):
        files = os.listdir(sub_dir)
        csv_files = [f for f in files if f.endswith('.csv')]
        json_files = [f for f in files if f.endswith('.json')]
        
        for csv_file in csv_files:
            file_path = os.path.join(sub_dir, csv_file)
            size = os.path.getsize(file_path) / 1024  # KB
            mtime = datetime.fromtimestamp(os.path.getmtime(file_path))
            print(f"   │   └── {csv_file} ({size:.1f} KB, {mtime.strftime('%Y-%m-%d %H:%M:%S')})")
        
        for json_file in json_files:
            file_path = os.path.join(sub_dir, json_file)
            size = os.path.getsize(file_path) / 1024  # KB
            mtime = datetime.fromtimestamp(os.path.getmtime(file_path))
            print(f"   │   └── {json_file} ({size:.1f} KB, {mtime.strftime('%Y-%m-%d %H:%M:%S')})")

# 3. 파이프라인 로그 확인
pipeline_pattern = "logs/pipeline/full_pipeline_*.log"
pipeline_logs = glob.glob(pipeline_pattern)
print(f"\n📋 파이프라인 로그 파일 ({len(pipeline_logs)}개):")
for log_file in sorted(pipeline_logs)[-3:]:  # 최근 3개만 표시
    size = os.path.getsize(log_file) / 1024  # KB
    mtime = datetime.fromtimestamp(os.path.getmtime(log_file))
    print(f"   └── {os.path.basename(log_file)} ({size:.1f} KB, {mtime.strftime('%Y-%m-%d %H:%M:%S')})")

# 4. 방금 실행된 파이프라인의 최신 파일 확인
print(f"\n🕒 최신 생성 파일 (방금 실행된 파이프라인):")
latest_infer_log = max(log_files, key=os.path.getmtime) if log_files else None
if latest_infer_log:
    size = os.path.getsize(latest_infer_log) / 1024
    mtime = datetime.fromtimestamp(os.path.getmtime(latest_infer_log))
    print(f"   📄 최신 추론 로그: {os.path.basename(latest_infer_log)} ({size:.1f} KB)")
    print(f"      └── 생성 시간: {mtime.strftime('%Y-%m-%d %H:%M:%S')}")

# 방금 생성된 제출 파일 찾기
all_csv_files = []
for sub_dir in submission_dirs:
    csv_files = glob.glob(os.path.join(sub_dir, "*.csv"))
    all_csv_files.extend(csv_files)

if all_csv_files:
    latest_csv = max(all_csv_files, key=os.path.getmtime)
    size = os.path.getsize(latest_csv) / 1024
    mtime = datetime.fromtimestamp(os.path.getmtime(latest_csv))
    print(f"   📊 최신 제출 파일: {os.path.basename(latest_csv)} ({size:.1f} KB)")
    print(f"      └── 생성 시간: {mtime.strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n✅ 파이프라인 통합 테스트 완료!")
print(f"📅 테스트 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 모든 구성 요소가 정상적으로 통합되어 작동함을 확인했습니다.")

🔍 파이프라인 실행 결과 파일 확인

📄 추론 로그 파일 (4개):
   └── infer_highperf_20250905_1734.log (1.5 KB, 2025-09-05 17:38:46)
   └── infer_highperf_20250905_2357.log (1.5 KB, 2025-09-06 00:03:31)
   └── infer_highperf_20250906_0014.log (1.5 KB, 2025-09-06 00:20:33)

📦 제출 파일 디렉토리들:
   ├── submissions/20250904
   │   └── infer.csv (71.0 KB, 2025-09-05 13:02:35)
   ├── submissions/20250905
   │   └── v09328-swin-highperf_ensemble_20250905_1714.csv (71.7 KB, 2025-09-05 17:38:46)
   ├── submissions/20250906
   │   └── v094-swin-highperf_ensemble_20250906_0014.csv (71.7 KB, 2025-09-06 00:20:33)

📋 파이프라인 로그 파일 (11개):
   └── full_pipeline_20250906_0011.log (0.7 KB, 2025-09-06 00:11:19)
   └── full_pipeline_20250906_0012.log (0.7 KB, 2025-09-06 00:12:24)
   └── full_pipeline_20250906_0014.log (1.4 KB, 2025-09-06 00:20:33)

🕒 최신 생성 파일 (방금 실행된 파이프라인):
   📄 최신 추론 로그: infer_highperf_20250906_0014.log (1.5 KB)
      └── 생성 시간: 2025-09-06 00:20:33
   📊 최신 제출 파일: v094-swin-highperf_ensemble_20250906_0014.csv (71.7 KB)
